In [ ]:
from CVRPSolver import solve, SMALL_CONFIGS, LARGE_CONFIGS
from random import randint, seed
from math import dist
import matplotlib.pyplot as plt

SEED = 0
seed(SEED)

: 

In [ ]:
def plot(routes, locations):
    def plot_route(route):
        x = [locations[loc][0] for loc in route]
        y = [locations[loc][1] for loc in route]
        plt.plot(x, y, marker='o', linestyle='-', color='b')

    for route in routes:
        plot_route(route)

    plt.xticks([])
    plt.yticks([])
    depot = locations[0]
    plt.scatter(*depot, color='r', s=100, marker='o', zorder=10)
    plt.show()


def scatter(locations):
    x = [loc[0] for loc in locations]
    y = [loc[1] for loc in locations]
    plt.scatter(x, y, marker='o', linestyle='-', color='b')
    plt.xticks([])
    plt.yticks([])
    depot = locations[0]
    plt.scatter(*depot, color='r', s=100, marker='o', zorder=10)
    plt.show()

: 

# Generation of a small CVRP

In [ ]:
number_of_locations = 35
number_of_vehicles = 15
vehicle_capacity = 100

locations = [(randint(0, 100), randint(0, 100)) for _ in range(number_of_locations)]
distance_matrix = [
    [dist(locations[i], locations[j]) for i in range(number_of_locations)]
    for j in range(number_of_locations)
]

demands = [
    randint(0, 3 * vehicle_capacity * number_of_vehicles // (2 * number_of_locations))
    for _ in range(number_of_locations)
]
scatter(locations)

: 

# Finding a solution using the CVRP solver
- The solver does not garantee an optimal solution, but the output of the solver is usually close to the optimal solution (within ~10% on the instances in `data`).

In [ ]:
sol = solve(distance_matrix, locations, demands, number_of_vehicles, vehicle_capacity, {"TIME_LIMIT": 30, "SEED": SEED, "TRIES": 1})
plot([[0] + route + [0] for route in sol], locations)

: 

# Performent configurations
I have compiled a list of `performent configurations` for small (less then around 100 locations) and larger problems (around 100 - 1000 locations) using [amltk](https://automl.github.io/amltk/latest/) and instances in `data`. You should try which configuration/s fits your problem the best. Configurations at the beginning of the lists should perform better.

In [ ]:
SAMPLE_CONFIG_SMALL = SMALL_CONFIGS[0] | {"TIME_LIMIT": 30, "SEED": SEED, "TRIES": 1}
sol = solve(distance_matrix, locations, demands, number_of_vehicles, vehicle_capacity, SAMPLE_CONFIG_SMALL)
plot([[0] + route + [0] for route in sol], locations)

: 

In [ ]:
SAMPLE_CONFIG_LARGE = LARGE_CONFIGS[0] | {"TIME_LIMIT": 30, "SEED": SEED, "TRIES": 1}
sol = solve(distance_matrix, locations, demands, number_of_vehicles, vehicle_capacity, SAMPLE_CONFIG_LARGE)
plot([[0] + route + [0] for route in sol], locations)

: 

# Parallel computing
For even better solutions, you should try more configurations with different seeds with time limit around 3-10 minutes (using pypy, probably a bit longer in python).

In [ ]:
import asyncio
from concurrent.futures import ProcessPoolExecutor
import nest_asyncio
nest_asyncio.apply()  # neccessary for running asyncio in jupyter notebook

: 

In [ ]:
N_WORKERS = 2
SAMPLE_CONFIGS = [
    config | {"TIME_LIMIT": 3*60, "SEED": seed}
    for config in SMALL_CONFIGS[:3]
    for seed in (0, 42)
]  # You should try even more configurations


def help(config):
    return solve(distance_matrix, locations, demands, number_of_vehicles, vehicle_capacity, config)


async def main(loop, n_workers):
    executor = ProcessPoolExecutor(max_workers=n_workers)
    return await asyncio.gather(*(loop.run_in_executor(executor, help, config) for config in SAMPLE_CONFIGS))


loop = asyncio.get_event_loop()
sol = min(loop.run_until_complete(main(loop, N_WORKERS)))
plot([[0] + route + [0] for route in sol], locations)


: 